# Function Description
This ipython file is for compress filters' port and protocol from range to prefix and re-print a new filter set. In default, the file structure should like that:

    ClassBench > code/ data/ output/

        data > fw_filters/ acl_filters/ ipc_filters/

        output > fw_filters/ acl_filters/ ipc_filters/
        
    *In which fw_filters means "fw filters"

This ipython file should be in code folder and filters should be put into data folder.

The compressed filters will be named as ".cmp"

In [3]:
def Prefix2Range(single_rule):
    # get a single rule of prefix form "0000*****" and change it to "xx : xx" form
    if '*' in single_rule:
        low_range = single_rule.replace('*','0')
        low_int = int(low_range,2)
        high_range = single_rule.replace('*','1')
        high_int = int(high_range,2)
        tmp_rule = "%d : %d"%(low_int,high_int)
    else:
        tmp_int = int(single_rule,2)
        tmp_rule = "%d : %d"%(tmp_int,tmp_int)
    return tmp_rule

def Range2Prefix(left,right,res,level = 0,mstr = '',vla = 0): 
    tmp_vla = 2**(15-level)+ vla
    if vla == left and right == vla + 2**(15-level+1)-1: # (00000,11111)
        res.append(mstr+'*'*(15-level+1))
        return
    if vla == left and right == tmp_vla - 1: # (00000,01111)
        res.append(mstr+'0'+'*'*(15-level))
        return
    if tmp_vla == left  and right == tmp_vla + 2**(15-level)-1: # (10000,11111)
        res.append(mstr+'1'+'*'*(15-level))
        return 
    if tmp_vla > right:
        tmp_str = mstr + '0'
        Range2Prefix(left,right,res,level+1,tmp_str,vla)
    elif tmp_vla <=left:
        tmp_str = mstr + '1'
        Range2Prefix(left,right,res,level+1,tmp_str,tmp_vla)
    else:
        tmp_str = mstr+'0'
        Range2Prefix(left,tmp_vla-1,res,level+1,tmp_str,vla)
        tmp_str = mstr+'1'
        Range2Prefix(tmp_vla,right,res,level+1,tmp_str,tmp_vla)

def ReadFile(file_path, rule = []):
    for x in open(file_path, 'r'):
        tmp = x.replace('@','').split('\t')
        rule.append(tmp[0:-1]) # remove the '\n'
    return rule

def ExpressRuleSet(rules, new_rules = []):
    for rule in rules:
        port = rule[2].replace(' ','').split(':') # port[0] is low and port[1] is high
        protocol = rule[3].replace(' ','').split(':') # protocol[0] is low and protocol[1] is high
        port_res = []
        Range2Prefix(int(port[0]), int(port[1]), port_res)
        proto_res = []
        Range2Prefix(int(protocol[0]), int(protocol[1]), proto_res)
        for single_port_res in port_res:
            for single_proto_res in proto_res:
                temp_port_range = Prefix2Range(single_port_res)
                temp_proto_range = Prefix2Range(single_proto_res)
                temp_rule = [rule[0],rule[1],temp_port_range,temp_proto_range,rule[4],rule[5]]
                # temp_rule = "@{0}\t{1}\t{2}\t{3}\t{4}\t\n".format(rule_test[0],rule_test[1],temp_port_range,temp_proto_range,rule_test[4],rule_test[5])
                new_rules.append(temp_rule)
    return new_rules
                
def PrintRuleSet(output_path, rules):
    output_file = open(output_path, 'w')
    with open(output_path, 'w') as f:
        for i in rules:
            print('@{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t'.format(i[0],i[1],i[2],i[3],i[4],i[5]), file=f)

def main(folder, filter_name):
    input_file = "../data/" + folder + "/" + filter_name +".txt"
    rules = ReadFile(input_file)
    new_rules = ExpressRuleSet(rules)
    output_file = "../output/" + folder + "/" + filter_name +".cmp"
    PrintRuleSet(output_file, new_rules)

In [4]:
fw_folder = "fw filters"
acl_folder = "acl filters"
ipc_folder = "ipc filters"
for i in range(10):
    filter_name = "MyFilters"+ "{}".format(i+1) + "k_1"
    main(fw_folder, filter_name)
    main(acl_folder, filter_name)
    main(ipc_folder, filter_name)